In [1]:
import numpy as np
import pandas as pd
from Drive import *
import AnalysisDataCleaner
import SupportFunctions

In [2]:
class Analysis:
    def __init__(self):
        self.fileDownloader()
        self.data=self.data_importer()
        self.data=AnalysisDataCleaner.data_cleaner(self.data)
        self.months=['January','February','March','April','May',\
                     'June','July','August','September','October',\
                     'November','December']
        
    def fileDownloader(self):
        """Downloads Student Gradebook.xlsx from GDrive"""
        try:
            auth=GDriveAuthenticator.GDriveAuthenticator('Drive\\client_secret.json')
            drive=Gdrive.Gdrive(auth.authenticate())
            drive.download()
        except:
            print('Problem in DRIVE API usage')
            raise
            
    def data_importer(self):
        """imports the data and merges into one dataframe and returns it"""
        excel_sheet=pd.read_excel('Student Gradebook.xlsx',sheet_name=None,usecols=range(0,10))
        list_data=[]
        for i in excel_sheet:
            list_data.append(excel_sheet[i])
        #data_july=excel_sheet['July'].copy()
        #data_august=excel_sheet['August'].copy()
        data=pd.concat(list_data).reset_index().drop(columns=['index'])
        #del data_july
        #del data_august
        return data
    
    def total_marks_leaderboard(self,month_name=None):
        """Returns a leaderboard of students containing data such as :- total, task, myday, knowledge sharing marks and
        count of late submissions and task won.
        Arguements:-
        :month_name - [Default: None] if any other value then returns leaderboard for the specified month. Make sure first
        letter of month is capital.

        Returns:
        tuple:- (dataframe object,dataframe with hidden indexes)"""
        data=self.data
        if month_name==None:
            total_marks=data.groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            total_marks.rename(columns={'Points':'Marks'})
            total_marks['Total Marks']=total_marks['Student'].apply(SupportFunctions.get_highest_marks,args=[data])
            total_marks['Task Marks']=total_marks['Student'].apply(SupportFunctions.get_task,args=[data])
            total_marks['MyDay Marks']=total_marks['Student'].apply(SupportFunctions.get_myday,args=[data])
            total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(SupportFunctions.get_gyan,args=[data])
            total_marks['Number of Late Submissions']=total_marks['Student'].apply(SupportFunctions.get_count_late_submission,args=[data])
            total_marks['Number of Tasks Won']=total_marks['Student'].apply(SupportFunctions.get_count_task_won,args=[data])
            return tuple((total_marks,total_marks.style.hide_index()))
        
        elif month_name in self.months:
            temp=data[data['Month']==month_name]
            total_marks=data[data['Month']==month_name].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            total_marks.rename(columns={'Points':'Marks'})
            total_marks['Total Marks']=total_marks['Student'].apply(SupportFunctions.get_highest_marks,args=[temp,month_name])
            total_marks['Task Marks']=total_marks['Student'].apply(SupportFunctions.get_task,args=[temp,month_name])
            total_marks['MyDay Marks']=total_marks['Student'].apply(SupportFunctions.get_myday,args=[temp,month_name])
            total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(SupportFunctions.get_gyan,args=[temp,month_name])
            total_marks['Number of Late Submissions']=total_marks['Student'].apply(SupportFunctions.get_count_late_submission,args=[temp,month_name])
            total_marks['Number of Tasks Won']=total_marks['Student'].apply(SupportFunctions.get_count_task_won,args=[temp,month_name])
            return tuple((total_marks,total_marks.style.hide_index()))
        
        else:
            raise ValueError('month_name has wrong spelling supplied')
            
    def student_task_marks(self,name=None,month_name=None):
        """Returns the task and points of the given student name in DataFrame
        Arguments:
            data: dataframe from which marks need to be extracted
            name: name of the student, only the first name and first letter capital.
            monthWise: default: False
                        set to True if dataframe needed only for a particular month
            month_name: default None.
                        if monthWise True and month_name is name of the month then
                        Dataframe of that particular month is given.

        Returns:
            tuple:- (dataframe object,dataframe with hidden indexes)
        """
        data=self.data
        if name==None:
            raise ValueError('Input a Name')
            
        if month_name!=None:
            if month_name in self.months:
                task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') &\
                                     (data['Student']==name) & \
                                     (data['Month']==month_name)]
                
            else:
                raise ValueError('Wrong month spelling given')
                
        elif month_name==None:
            task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') &\
                                 (data['Student']==name)]
        task_marks=task_marks[['Date','Month','Task','Student','Points','Total','Late Submission','Task Winner']]
        task_marks['Late Submission'].replace([0,1],['No','Yes'],inplace=True)
        task_marks['Task Winner'].replace([0,1],['No','Yes'],inplace=True)
        return tuple((task_marks,task_marks.style.hide_index()))
    
    def student_myday(self,name=None,month_name=None):
        """Gets the myday marks of the given student
        Arguements:
        name - name of the student
        month_name - [Default: None] name of month to check the dataframe for.

        Returns:
        tuple:- (dataframe object,dataframe with hidden indexes)"""
        data=self.data
        myday='AjKyaUkhada'
        if name==None:
            raise ValueError('Input a name')
        if month_name==None:
            myday_marks=data[(data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].\
                sum().sort_values(ascending=False).reset_index()
            myday_marks.rename(columns={'Points':'Myday Marks'},inplace=True)
            myday_count=data[(data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].\
                count().sort_values(ascending=False).reset_index()
            myday_count.rename(columns={'Points':'Number of days'},inplace=True)
            myday_data=myday_marks.merge(myday_count,on='Student')
            return tuple((myday_data,myday_data.style.hide_index()))
        elif month_name in self.months:
            myday_marks=data[(data['Month']==month_name) & (data['Task']==myday) &\
                             (data['Student']==name)].groupby('Student')['Points'].\
                            sum().sort_values(ascending=False).reset_index()
            myday_marks.rename(columns={'Points':'Myday Marks'},inplace=True)
            myday_count=data[(data['Month']==month_name) & (data['Task']==myday) & \
                             (data['Student']==name)].groupby('Student')['Points'].\
                            count().sort_values(ascending=False).reset_index()
            myday_count.rename(columns={'Points':'Number of days'},inplace=True)
            myday_data=myday_marks.merge(myday_count,on='Student')
            return tuple((myday_data,myday_data.style.hide_index()))

    def student_knowledge_marks(self,name=None,month_name=None):
        """Gets the knowledge sharing marks of the given student
        Arguements:
        name - name of the student
        month_name - [Default: None] name of month to check the dataframe for.

        Returns:
        tuple:- (dataframe object, dataframe with hidden indexes)
        """
        data=self.data
        knowledge='Knowledge Sharing'
        if name==None:
            raise ValueError('Input a name')
        if month_name==None:
            knowledge_marks=data[(data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            knowledge_marks.rename(columns={'Points':'Knowledge Sharing Marks'},inplace=True)
            knowledge_count=data[(data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
            knowledge_count.rename(columns={'Points':'Number of days'},inplace=True)
            knowledge_data=knowledge_marks.merge(knowledge_count,on='Student')
            return tuple((knowledge_data,knowledge_data.style.hide_index()))
        elif month_name in self.months:
            knowledge_marks=data[(data['Month']==month_name) & (data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
            knowledge_marks.rename(columns={'Points':'Knowledge Sharing Marks'},inplace=True)
            knowledge_count=data[(data['Month']==month_name) & (data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
            knowledge_count.rename(columns={'Points':'Number of days'},inplace=True)
            knowledge_data=knowledge_marks.merge(knowledge_count,on='Student')
            return tuple((knowledge_data,knowledge_data.style.hide_index()))

In [3]:
a=Analysis()

Download. 100


In [4]:
a.total_marks_leaderboard()[0]

,Student,Points,Total Marks,Task Marks,MyDay Marks,Knowledge Sharing Marks,Number of Late Submissions,Number of Tasks Won
0,Kunal,352.5,376,241.5,105.0,6.0,5,4
1,Ujjainee,346.0,378,233.0,103.0,10.0,4,0
2,Shakib,340.0,373,232.0,99.0,9.0,4,3
3,Surabhi,337.0,371,231.0,103.0,3.0,4,1
4,Purbita,335.0,369,231.0,95.0,9.0,6,2
5,Vishal,327.5,364,228.5,97.0,2.0,2,2
6,Chandrima,325.0,377,213.0,101.0,11.0,6,3
7,Durga,325.0,378,222.0,98.0,5.0,6,1
8,Siddhishikha,323.5,372,226.5,90.0,7.0,8,0
9,Dipam,323.5,371,217.5,103.0,3.0,2,1


In [5]:
a.total_marks_leaderboard(month_name='September')[0]

,Student,Points,Total Marks,Task Marks,MyDay Marks,Knowledge Sharing Marks,Number of Late Submissions,Number of Tasks Won
0,Durga,124.5,135,99.5,24.0,1.0,3,0
1,Dipam,122.0,125,92.0,27.0,3.0,2,1
2,Siddhishikha,121.5,130,96.5,22.0,3.0,4,0
3,Shakib,120.5,125,90.5,27.0,3.0,2,1
4,Kunal,120.0,124,91.0,27.0,2.0,2,2
5,Chandrima,120.0,127,88.0,27.0,5.0,3,2
6,Purbita,118.5,125,88.5,25.0,5.0,3,0
7,Sonali,118.0,125,88.0,27.0,3.0,2,1
8,Ujjainee,117.5,126,86.5,27.0,4.0,3,0
9,Surabhi,115.5,123,87.5,27.0,1.0,3,0


In [6]:
a.student_task_marks('Kunal')[0]

,Date,Month,Task,Student,Points,Total,Late Submission,Task Winner
5,22,July,Linkedin Profile create/update,Kunal,8.0,10,No,No
45,23,July,Blog on Machine Learning and it's impact,Kunal,21.0,25,No,No
155,30,July,Blog on Different data based job roles,Kunal,21.5,25,Yes,Yes
195,31,July,Portfolio home page,Kunal,12.0,15,Yes,No
246,1,August,Video Resume,Kunal,20.0,20,No,No
284,8,March,Online Resume,Kunal,18.0,20,No,No
316,8,April,Blog and Project Pages,Kunal,9.5,10,No,No
523,13,August,Github Profile Creation,Kunal,9.5,10,No,Yes
568,14,August,PPT Submission,Kunal,5.0,5,Yes,No
766,23,August,Blog on Data Analysis Process,Kunal,26.0,30,No,No


In [7]:
a.student_task_marks('Kunal','September')[0]

,Date,Month,Task,Student,Points,Total,Late Submission,Task Winner
1162,11,September,Web Scraping,Kunal,10.0,10,No,Yes
1296,17,September,Data from API,Kunal,15.0,15,Yes,No
1328,17,September,Batsman V Bowler,Kunal,20.0,20,No,Yes
1375,17,September,Start/Country Dataset,Kunal,46.0,50,Yes,No


In [8]:
a.student_myday('Kunal')[0]

,Student,Myday Marks,Number of days
0,Kunal,105.0,57


In [9]:
a.student_myday('Kunal',month_name='September')[0]

,Student,Myday Marks,Number of days
0,Kunal,27.0,18


In [10]:
a.student_knowledge_marks('Kunal')[0]

,Student,Knowledge Sharing Marks,Number of days
0,Kunal,6.0,3


In [11]:
a.student_knowledge_marks('Kunal',month_name='August')[0]

,Student,Knowledge Sharing Marks,Number of days
0,Kunal,4.0,2


In [12]:
a.data

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Month,Year,DayName
0,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,July,2019,Monday
1,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,July,2019,Monday
2,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,July,2019,Monday
3,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,July,2019,Monday
4,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,July,2019,Monday
5,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kunal,0,8.0,10,0,July,2019,Monday
6,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Prasoon,0,5.0,10,0,July,2019,Monday
7,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Purbita,0,6.0,10,0,July,2019,Monday
8,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Roumyak,0,6.0,10,0,July,2019,Monday
9,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Shakib,0,6.0,10,0,July,2019,Monday
